In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import csv
with open("/kaggle/input/ubiquant-market-prediction/train.csv") as f:
    reader = csv.reader(f)
    cols = next(reader)

In [ ]:
col_dtype = dict()
for col in cols:
    col_dtype[col] = "float16"

col_dtype['row_id'] = "object"
col_dtype['time_id'] = "uint16"
col_dtype['investment_id'] = "uint16"

In [ ]:
train_df = pd.read_csv("/kaggle/input/ubiquant-market-prediction/train.csv", dtype=col_dtype)
train_df

In [ ]:
investments = train_df.groupby("investment_id")

In [ ]:
look_back = 20

from collections import defaultdict
test_input = defaultdict(lambda: np.zeros(look_back, dtype='float16'))

for investment in investments:
    last_section = np.array(investment[1].iloc[-look_back:, 3])
    test_input[investment[0]][-len(last_section):] = last_section

In [ ]:
import ubiquant
env = ubiquant.make_env()   # initialize the environment
iter_test = env.iter_test()    # an iterator which loops over the test set and sample submission
for (test_df, sample_prediction_df) in iter_test:
    
    X = test_df["investment_id"].apply(lambda x: pd.Series(test_input[x]))
    pred = X.mean(axis=1)
    for index, row in test_df.iterrows():
        test_input[row["investment_id"]][:-1] = X.loc[index].iloc[1:]
        test_input[row["investment_id"]][-1] = pred[index]

    sample_prediction_df['target'] = pred  # make your predictions here
    env.predict(sample_prediction_df)   # register your predictions

In [ ]:
pd.read_csv("submission.csv")